# This notebook enables comparing stanza and YAP for Hebrew text analysis

In [4]:
import sys
import pandas as pd
import stanza
stanza.download('he')
import requests
import json
from time import sleep
from pandas.io.json import json_normalize
sys.path.append("C:\HNLP\Ex3\YAP-Wrapper-master")
from yap_api import YapApi

2021-05-15 14:08:27 INFO: Downloading default packages for language: he (Hebrew)...
2021-05-15 14:09:58 INFO: Finished downloading models and saved to C:\Users\yelfs\stanza_resources.


In [17]:
class Processor:
    def __init__(self):
        self.heb_nlp = stanza.Pipeline(lang='he', processors='tokenize,mwt,pos,lemma,depparse')
        #replace MY_TOKEN with the token you got from the langndata website
        self.yap_token="a073769bd24ead12455d64d220c0287b"
        self.ip='127.0.0.1:8000'
        self.yap=YapApi() 
    
    def print_stanza_analysis(self, text, name):
        text += " XX"
        doc=self.heb_nlp(text)
        lst=[]
        for sen in doc.sentences:
            for token in sen.tokens:
                for word in token.words:
                    features=[(word.text,
                               word.lemma,
                               word.upos,
                               word.xpos,
                               word.head,
                               word.deprel,
                               word.feats)]

                    df=pd.DataFrame(features, columns=["text", "lemma", "upos", "xpos", "head", "deprel","feats"])
                    lst.append(df)
        tot_df=pd.concat(lst, ignore_index=True)
        tot_df=tot_df.shift(1).iloc[1:]
        tot_df["head"]=tot_df["head"].astype(int)
        tot_df.head(50).to_csv(name, encoding="utf-8")
        print(tot_df.head(50))
        
    def print_yap_analysis(self, text, name):
        text= text.replace(r'"', r'\"')
        tokenized_text, segmented_text, lemmas, dep_tree, md_lattice, ma_lattice=self.yap.run(text, self.ip)                 
        dep_tree.set_index("num",inplace=True)
        dep_tree.to_csv(name, encoding="utf-8")
        print(dep_tree)
 

In [20]:
texts = [
    """
    המשפט שאנחנו המצאנו.
    """
    ]
for index, text in enumerate(texts):
    processor=Processor()
    print("stanza")
    name = str(index) + '_STA.csv'
    processor.print_stanza_analysis(text, name)
    print("------------------------")
    print("yap")
    name = str(index) + '_YAP.csv'
    processor.print_yap_analysis(text, name)

2021-05-15 17:55:05 INFO: Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2021-05-15 17:55:05 INFO: Use device: cpu
2021-05-15 17:55:05 INFO: Loading: tokenize
2021-05-15 17:55:05 INFO: Loading: mwt
2021-05-15 17:55:05 INFO: Loading: pos
2021-05-15 17:55:05 INFO: Loading: lemma
2021-05-15 17:55:05 INFO: Loading: depparse
2021-05-15 17:55:06 INFO: Done loading processors!


stanza
    text lemma   upos   xpos  head     deprel  \
1      ה     ה    DET    DET     2    det:def   
2   משפט  משפט   NOUN   NOUN     0       root   
3      ש     ש  SCONJ  SCONJ     6       mark   
4  אנחנו   הוא   PRON   PRON     6      nsubj   
5      ה     ה    DET    DET     6       mark   
6  מצאנו   מצא   VERB   VERB     2  acl:relcl   
7      .     .  PUNCT  PUNCT     2      punct   

                                               feats  
1                                       PronType=Art  
2                            Gender=Masc|Number=Sing  
3                                               None  
4  Gender=Fem,Masc|Number=Plur|Person=1|PronType=Prs  
5                                       PronType=Art  
6  Gender=Fem,Masc|HebBinyan=PAAL|Number=Plur|Per...  
7                                               None  
------------------------
yap
Start Yap call
Tokens: 4
End Yap call 0 /0
      word lemma    pos  pos_2  \
num                              
1        ה     ה    